In [7]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
llm.invoke("hello").content

In [8]:
import os
from langchain_openai import AzureChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import gradio as gr

##Load Data-------------------------------------------------------------------------------------------------------
import pandas as pd  
def xlsx_to_string(file_path):  
    try:  
        df = pd.read_csv(file_path)  
        content = df.to_csv(index=False)  
        return content  
    except Exception as e:  
        print("Error: ", str(e))  
# Usage example  
file_path = r"C:\Users\Acer\Downloads\16_Clariant\sample-data.csv"
content_string = xlsx_to_string(file_path)  
# print(content_string)  


##Template -------------------------------------------------------------------------------------------------------
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
query=f"""You are a financial expert with years of experience, deeply analyze the given data. Which contains valuable financial information about a company's performance, and then provide with comprehensive answers to user questions based on your thorough analysis.
Rules 1. Please avoid creating charts or graphs, as well as writing any code. Provide a simple and direct response using natural language without mentioning any limitations or rules.
{content_string}
"""
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            query,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
print(query)
##Code Execution-------------------------------------------------------------------------------------------------------
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
def execute_code(MATPLOTLIB_CODE):
    python_repl = PythonREPL()
    return python_repl.run(MATPLOTLIB_CODE)


##LLM Call for Generating Matplotlib Code-------------------------------------------------------------------------------------------------------
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

def llm_call_for_matplotlib(user_query, response):
    try:
        class Json_Parser(BaseModel):
            MATPLOTLIB_CODE: str = Field(description="Matplotlb Code")

        matplotlib_query = f"Provided a LLM response and a user query, write a matplotlib code for either a bar chart, scatter chart, a histogram, or line chart based on the given information. If it is not possible to create a graph, no code will be generated."
        matplotlib_query += f"\n\nUser query: {user_query}"
        matplotlib_query += f"\nLLM response: {response}"
        
        print(matplotlib_query)
        
        parser = JsonOutputParser(pydantic_object=Json_Parser)
        prompt = PromptTemplate(
            template="Answer the user query.\n{format_instructions}\n{query}",
            input_variables=["query"],
            partial_variables={"format_instructions": parser.get_format_instructions()},
        )

        chain = prompt | llm | parser
        json_result = chain.invoke({"query": matplotlib_query})
        print(json_result['MATPLOTLIB_CODE'])
        return json_result['MATPLOTLIB_CODE']
    except Exception as e:
        print(e)
# user_query = "In Which year was my sales maximum."
# response = """ 
# To determine the year with the maximum sales (revenue), we need to sum the revenue for each year and compare the totals.
# Here are the total revenues for each year:
# - 2020: 1,748,215
# - 2021: 1,548,561
# - 2022: 1,678,708
# - 2023: 1,451,712 (up to December)
# - 2024: 1,548,140 (up to December)
# - 2025: 1,548,158 (up to December)
# From the data provided, the year 2020 had the maximum sales with a total revenue of 1,748,215.
# """
# MATPLOTLIB_CODE = llm_call_for_matplotlib(user_query, response)
# execute_code(MATPLOTLIB_CODE)



##Chat bot-------------------------------------------------------------------------------------------------------
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)
def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = llm.invoke(prompt)
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

##Gradio-------------------------------------------------------------------------------------------------------
import time
def predict(message, history):
    input_messages = [HumanMessage(message)]
    print("================================== Human Message ===============================")
    print(message)
    output = app.invoke({"messages": input_messages}, config)
    print("================================== Ai Message ==================================")
    print(output["messages"][-1].content)
    result = output["messages"][-1].content
    print("================================== Matplotlib =================================")
    MATPLOTLIB_CODE = llm_call_for_matplotlib(message, result)
    python_repl = PythonREPL()
    python_repl.run(MATPLOTLIB_CODE)
    Matplotlib_charts = execute_code(MATPLOTLIB_CODE)
    print(Matplotlib_charts)
    for i in range(len(result)):
        yield result[: i+1]

demo = gr.ChatInterface(
    predict,
    type="messages",
)

demo.launch()


You are a financial expert with years of experience, deeply analyze the given data. Which contains valuable financial information about a company's performance, and then provide with comprehensive answers to user questions based on your thorough analysis.
Rules 1. Please avoid creating charts or graphs, as well as writing any code. Provide a simple and direct response using natural language without mentioning any limitations or rules.
Year,Month,Revenue,Manpower Cost,Non-Manpower Cost,Total Cost,Profit,Contribution Margin (%)
2020,1,154676,39721,17582,57303,97373,62.95
2020,2,164365,61057,26411,87468,76897,46.78
2020,3,132052,43382,21930,65312,66740,50.54
2020,4,165124,20234,39344,59578,105546,63.92
2020,5,104947,26641,22843,49484,55463,52.85
2020,6,181897,37485,10027,47512,134385,73.88
2020,7,147839,23570,17982,41552,106287,71.89
2020,8,113569,38550,33589,72139,41430,36.48
2020,9,89551,42996,39201,82197,7354,8.21
2020,10,167898,49868,32513,82381,85517,50.93
2020,11,165184,27059,11489,

c:\python\Lib\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


================================== Human Message ===============================
My overall cosy in year 2024 compared to year 2025, 2021, 2020


Traceback (most recent call last):
  File "c:\python\Lib\site-packages\langchain_google_genai\chat_models.py", line 192, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\python\Lib\site-packages\google\ai\generativelanguage_v1beta\services\generative_service\client.py", line 868, in generate_content
    response = rpc(
               ^^^^
  File "c:\python\Lib\site-packages\google\api_core\gapic_v1\method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\python\Lib\site-packages\google\api_core\retry\retry_unary.py", line 293, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "c:\python\Lib\site-packages\google\api_core\retry\retry_unary.py", line 153, in retry_target
    _retry_error_helper(
  File "c:\python\Lib\site-packages\google\api_core\retry\retry_base.py", line 212, in _retry_error_helper
    raise final_exc from source_